# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
weather_metadata = "output_data/my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hanzhong,33.0728,107.0303,59.23,56,4,3.18,CN,1613098800
1,1,La Ronge,55.1001,-105.2842,-13.97,76,100,10.33,CA,1613098800
2,2,Souillac,-20.5167,57.5167,78.60,74,87,11.41,MU,1613098800
3,3,Kibungan,16.6950,120.6553,73.96,78,27,4.27,PH,1613098800
4,4,Provideniya,64.3833,-173.3000,-7.96,73,55,19.77,RU,1613098800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#weatherdf = weatherdf.dropna()

hum_rate = weatherdf["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,Dwārka,22.2394,68.9678,70.86,71,0,7.67,IN,1613098800
121,121,Busselton,-33.6500,115.3333,70.03,56,0,5.53,AU,1613098800
123,123,Lisala,2.1500,21.5167,71.10,42,0,1.32,CD,1613098800
131,131,Atuona,-9.8000,-139.0333,79.59,74,0,9.91,PF,1613098800
185,185,Salalah,17.0151,54.0924,71.38,38,0,6.87,OM,1613098800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
#hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng
12,Dwārka,IN,22.2394,68.9678
121,Busselton,AU,-33.6500,115.3333
123,Lisala,CD,2.1500,21.5167
131,Atuona,PF,-9.8000,-139.0333
185,Salalah,OM,17.0151,54.0924


In [8]:

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 12: Dwārka.
Closest hotel to Dwārka at 22.2394 , 68.9678 is Hotel Jayanti.
------------
Retrieving Results for Index 121: Busselton.
Closest hotel to Busselton at -33.65 , 115.3333 is Gale Street Motel & Villas.
------------
Retrieving Results for Index 123: Lisala.
Closest hotel to Lisala at 2.15 , 21.5167 is Mg Bokula Centre de Santé de Référence.
------------
Retrieving Results for Index 131: Atuona.
Closest hotel to Atuona at -9.8 , -139.0333 is Pension Temetiu Village.
------------
Retrieving Results for Index 185: Salalah.
Closest hotel to Salalah at 17.0151 , 54.0924 is Alrehab Palace 1.
------------
Retrieving Results for Index 315: Myitkyina.
Closest hotel to Myitkyina at 25.3833 , 97.4 is AYA Bank.
------------
Retrieving Results for Index 533: Kishanganj.
Closest hotel to Kishanganj at 26.1167 , 87.9333 is HOTEL SILVERSTAR.
------------
Retrieving Results for Index 560: Kaeo.
Closest hotel to Kaeo at -35.1 , 173.7833 is Ascension Creative: Graphi

In [9]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
12,Dwārka,IN,22.2394,68.9678,Hotel Jayanti
121,Busselton,AU,-33.6500,115.3333,Gale Street Motel & Villas
123,Lisala,CD,2.1500,21.5167,Mg Bokula Centre de Santé de Référence
131,Atuona,PF,-9.8000,-139.0333,Pension Temetiu Village
185,Salalah,OM,17.0151,54.0924,Alrehab Palace 1


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map


# Display figure


In [12]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius =1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

ValueError: weights must be of the same length as locations or None